In [0]:
import pandas as pd
import numpy as np
import cv2
from keras.utils import Sequence
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras.optimizers import RMSprop
from keras import backend as K
from keras import Sequential

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
#!pip install tensorflow-gpu

# Seen Dataset

In [0]:
# ENV VARS
stickyPath = "../../dataset/seen-dataset/TrainingSet/"

In [0]:
lookupSeenTrain = pd.read_csv("../../dataset/seen-dataset/dataset_seen_training_siamese.csv")
lookupSeenTrain.sample(frac=1).reset_index(drop=True)
lookupSeenValidate = pd.read_csv("../../dataset/seen-dataset/dataset_seen_validation_siamese.csv")
lookupSeenValidate.sample(frac=1).reset_index(drop=True)
inputShape = np.shape(cv2.imread("../../dataset/seen-dataset/TrainingSet/0001a_num4.png",cv2.IMREAD_GRAYSCALE))

In [0]:
testImage = cv2.imread("../../dataset/seen-dataset/TrainingSet/0001a_num4.png",cv2.IMREAD_GRAYSCALE)

In [0]:
testImage.reshape((64,64,1))

In [0]:
hshift = (-64,64)
vshift = (-64,64)

In [0]:
randH = np.random.randint(hshift[0],hshift[1])
randV = np.random.randint(vshift[0],vshift[1])

In [0]:
testImageShiftedH=np.roll(axis=0,a=testImage,shift=randH)

In [0]:
testImageShiftedV=np.roll(axis=0,a=testImage,shift=randV)

In [0]:
class genDataSeen(Sequence):
    def __init__(self,lookupSeenTrain,batchSize,action,flag):
        self.lookupSeenTrain, self.batchSize = lookupSeenTrain,batchSize
        self.action = action
        self.hshift = (-64,64)
        self.vshift = (-64,64)
        self.flag = flag
        if(action=='train'):
            self.stickyPath = "../../dataset/seen-dataset/TrainingSet/"
        if(action=='validate'):
            self.stickyPath = "../../dataset/seen-dataset/ValidationSet/"
    
    def __len__(self):
        return (int(np.floor(len(self.lookupSeenTrain) / float(self.batchSize))))
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = lookupSeenTrain.sample(frac=1).reset_index(drop=True)
    
    def __getitem__(self,idx):
        leftImages = list(self.lookupSeenTrain.left[idx*self.batchSize:(idx+1)*self.batchSize])
        rightImages = list(self.lookupSeenTrain.right[idx*self.batchSize:(idx+1)*self.batchSize])
        labels = list(self.lookupSeenTrain.label[idx*self.batchSize:(idx+1)*self.batchSize])
        
        skeleton = []
        for image in range(self.batchSize):
            leftImage = cv2.imread(str(self.stickyPath+
                                       leftImages[image]),
                                   cv2.IMREAD_GRAYSCALE)
            rightImage = cv2.imread(str(self.stickyPath+
                                       rightImages[image]),
                                   cv2.IMREAD_GRAYSCALE)
            # Floating Point
            leftImage = leftImage.astype('float32')
            leftImage /= 255
            rightImage = rightImage.astype('float32')
            rightImage /= 255
            
            # Shifting
            randH = np.random.randint(self.hshift[0],self.hshift[1])
            randV = np.random.randint(self.vshift[0],self.vshift[1])
            
            leftImageShiftedH=np.roll(axis=0,a=leftImage,shift=randH)
            rightImageShiftedH=np.roll(axis=0,a=rightImage,shift=randH)
            
            leftImageShiftedV=np.roll(axis=0,a=leftImage,shift=randV)
            rightImageShiftedV=np.roll(axis=0,a=rightImage,shift=randV)

            # Random Swap
            if(int(np.random.binomial(1, 0.5, 1)) == 1):
                temp = [rightImageShiftedV,leftImageShiftedH]
            else:
                temp = [leftImageShiftedH,rightImageShiftedV]
            skeleton.append(temp)

        skeleton = np.array(skeleton)
        left = []
        left.append(skeleton[:,0,:,:])
        left = np.array(left)
        right = []
        right.append(skeleton[:,1,:,:])
        right = np.array(right)
        if(self.flag=='cnn'):
            left = left.reshape((batchSize,64,64,1))
            right = right.reshape((batchSize,64,64,1))
            return ([left,right],np.array(labels))
        else:
            return ([skeleton[:,0,:,:],skeleton[:,1,:,:]],np.array(labels))

## Baseline Model

In [0]:
def create_base_network(input_shape):
    '''
    Base network to be shared (eq. to feature extraction).
    Ref: Keras Documentation on MNNIST Siamese
    '''
    _input = Input(shape=input_shape)
    x = Flatten()(_input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    return Model(_input, x)

In [0]:
base_network = create_base_network(inputShape)

In [0]:
inputLeft = Input(shape=inputShape)
inputRight = Input(shape=inputShape)

In [0]:
processed_left = base_network(inputLeft)
processed_right = base_network(inputRight)

In [0]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [0]:
distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_left, processed_right])

In [0]:
model = Model([inputLeft, inputRight], distance)

In [0]:
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])

In [0]:
batchSize = 125
numEpochs = 50
initialEpoch = 0

In [0]:
trainingBatchGen = genDataSeen(lookupSeenTrain,batchSize,"train")
validateBatchGen = genDataSeen(lookupSeenValidate,batchSize,"validate")

In [0]:
model.fit_generator(generator=trainingBatchGen,
                    steps_per_epoch=(len(lookupSeenTrain) // batchSize),
                    epochs=numEpochs,
                    verbose=1,
                    validation_data=validateBatchGen,
                    validation_steps=(len(lookupSeenValidate) // batchSize),
                    use_multiprocessing=True,
                    initial_epoch=0,
                    workers=32,
                    max_queue_size=32)

In [0]:
inputShape

In [0]:
def caffeReplicatedModel(input_shape):
    model = Sequential()
    model.add(Conv2D(filters=20,kernel_size=5,
                     kernel_initializer="glorot_normal",
                     strides=1,activation="relu",
                     use_bias=True,
                     bias_initializer="glorot_normal",
                     input_shape=(64, 64, 1)))
    model.add(MaxPooling2D(pool_size=2,strides=2))
    model.add(Conv2D(filters=20,kernel_size=5,
                     kernel_initializer="glorot_normal",
                     strides=1,activation="relu",
                     use_bias=True,
                     bias_initializer="glorot_normal",
                     input_shape=(64, 64, 1)))
    model.add(MaxPooling2D(pool_size=2,strides=2))
    model.add(Flatten())
    return model

In [0]:
inputShape = (64, 64, 1)

In [0]:
def caffeReplicatedModel(input_shape):
    _input = Input(input_shape)
    x = Conv2D(filters=20,kernel_size=5,
                     kernel_initializer="glorot_normal",
                     strides=1,activation="relu",
                     use_bias=False)(_input)
    x = MaxPooling2D(pool_size=2,strides=2)(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=20,kernel_size=5,
                     kernel_initializer="glorot_normal",
                     strides=1,activation="relu",
                     use_bias=False)(x)
    x = MaxPooling2D(pool_size=2,strides=2)(x)
    x = Flatten()(x)
    return Model(_input, x)

In [0]:
nw2 = caffeReplicatedModel(inputShape)

In [0]:
inputLeft = Input(shape=inputShape)
inputRight = Input(shape=inputShape)
processed_left = nw2(inputLeft)
processed_right = nw2(inputRight)

In [0]:
model = Model([inputLeft, inputRight], distance)

In [0]:
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])

In [0]:
batchSize = 25
numEpochs = 2
initialEpoch = 0

In [0]:
trainingBatchGen = genDataSeen(lookupSeenTrain,batchSize,"train","cnn")
validateBatchGen = genDataSeen(lookupSeenValidate,batchSize,"validate","cnn")

In [0]:
model.fit_generator(generator=trainingBatchGen,
                    steps_per_epoch=(len(lookupSeenTrain) // batchSize),
                    epochs=numEpochs,
                    verbose=1,
                    validation_data=validateBatchGen,
                    validation_steps=(len(lookupSeenValidate) // batchSize),
                    use_multiprocessing=True,
                    initial_epoch=0,
                    workers=64,
                    max_queue_size=32)

Epoch 1/50
  4/228 [..............................] - ETA: 27:37 - loss: 0.0425 - accuracy: 0.9520

Process ForkPoolWorker-248:
Process ForkPoolWorker-252:
Process ForkPoolWorker-194:
Process ForkPoolWorker-202:
Process ForkPoolWorker-231:
Process ForkPoolWorker-237:
Process ForkPoolWorker-227:
Process ForkPoolWorker-221:
Process ForkPoolWorker-250:
Process ForkPoolWorker-211:
Process ForkPoolWorker-256:
Process ForkPoolWorker-245:
Process ForkPoolWorker-215:
Process ForkPoolWorker-209:
Process ForkPoolWorker-218:
Process ForkPoolWorker-232:
Process ForkPoolWorker-223:
Process ForkPoolWorker-219:
Process ForkPoolWorker-243:
Process ForkPoolWorker-204:
Process ForkPoolWorker-247:
Process ForkPoolWorker-196:
Process ForkPoolWorker-225:
Process ForkPoolWorker-217:
Process ForkPoolWorker-253:
Process ForkPoolWorker-195:
Process ForkPoolWorker-222:
Process ForkPoolWorker-244:
Process ForkPoolWorker-193:
Process ForkPoolWorker-241:
Process ForkPoolWorker-203:
Process ForkPoolWorker-210:
Process ForkPoolWorker-233:
Process ForkPoolWorker-200:
Process ForkPoolWorker-207:
Process ForkPoolWork

Traceback (most recent call last):
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwar

  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pratik/anaconda3/envs/

  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing

  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
KeyboardInterrupt
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rloc

  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
KeyboardInterrupt
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/pratik/anaconda3/envs/aml/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()

KeyboardInterrupt: 